# Lab 2 - Widgets in Shiny 

In this lab notebook, we will see how to add control widgets to our Shiny apps. 
A widget is a web element that users can interact with. Widgets provide your users a way to send messages to the Shiny app. 
Users can enter text, choose options, and click on buttons to interact with the visualization using the widgets. 

Here are the basic control widgets in Shiny: 

<img src="../images/basicwidgets.png">

---


Shiny comes with a family of pre-built widgets, each created with a transparently named R function. 
For example, Shiny provides a function named ```actionButton``` that creates an Action Button and a function named ```sliderInput``` that creates a slider bar.

The standard Shiny widgets are:

| Widget              | Use           |
|---------------------|---------------|
| actionButton        | Action Button |
| checkboxGroupInput  | A group of check boxes |
| checkboxInput       | A single check box |
| dateInput           | A calendar to aid date selection |
| dateRangeInput      | A pair of calendars for selecting a date range |
| fileInput           | A file upload control wizard                   |
| helpText            | Help text that can be added to an input form   |
| numericInput        | A field to enter numbers                       |
| radioButtons        | A set of radio buttons                         |
| selectInput         | A box with choices to select from    |
| sliderInput         | A slider bar |
| submitButton        | A submit button |
| textInput           | A field to enter text |


To add a widget to your app, place a widget function in sidebarPanel or mainPanel in your ```ui``` object. 
Each widget function requires several arguments. The first two arguments for each widget are:

**a name for the widget:** The user will not see this name, but you use it to access the widget’s value. The name should be a character string.

**a label:** This label will appear with the widget in your app. It should be a character string, but it can be an empty string "".

For example, 

```R
actionButton("action", label = "Action")
``` 

Here, the **name** is “action” and the **label** is “Action”. 
"action" is the name *you use to acces this widget* inside your program.  
"Action" is just a string that is displayed on the button. 

The remaining arguments vary from widget to widget, depending on what the widget needs to do its job. 
They include things like initial values, ranges, and increments. 

The following code recreates the layout above. **Pay attention** to how the layout is given by rows and columns, and how choices are given as a list of string and numeric values: 

---
```R

library(shiny)

# UI

ui <- fluidPage(
  titlePanel("Basic widgets"),
  
  fluidRow(
    
    column(3,
           h3("Buttons"),
           actionButton("action", "Action"),
           br(),
           br(), 
           submitButton("Submit")),  
    column(3,
           h3("Single checkbox"),
           checkboxInput("checkbox", "Choice A", value = TRUE)),  
    column(3, 
           checkboxGroupInput("checkGroup", 
                              h3("Checkbox group"), 
                              choices = list("Choice 1" = 1, 
                                             "Choice 2" = 2, 
                                             "Choice 3" = 3),
                              selected = 1)),  
    column(3, 
           dateInput("date", 
                     h3("Date input"), 
                     value = "2014-01-01"))   
  ),
  
  fluidRow(
    
    column(3,
           dateRangeInput("dates", h3("Date range"))),  
    column(3,
           fileInput("file", h3("File input"))),  
    column(3, 
           h3("Help text"),
           helpText("Note: help text isn't a true widget,", 
                    "but it provides an easy way to add text to",
                    "accompany other widgets.")),  
    column(3, 
           numericInput("num", 
                        h3("Numeric input"), 
                        value = 1))   
  ),
  
  fluidRow(
    
    column(3,
           radioButtons("radio", h3("Radio buttons"),
                        choices = list("Choice 1" = 1, "Choice 2" = 2,
                                       "Choice 3" = 3),selected = 1)),  
    column(3,
           selectInput("select", h3("Select box"), 
                       choices = list("Choice 1" = 1, "Choice 2" = 2,
                                      "Choice 3" = 3), selected = 1)),  
    column(3, 
           sliderInput("slider1", h3("Sliders"),
                       min = 0, max = 100, value = 50),
           sliderInput("slider2", "",
                       min = 0, max = 100, value = c(25, 75))
    ),  
    column(3, 
           textInput("text", h3("Text input"), 
                     value = "Enter text..."))   
  )
)

# server
server <- function(input, output) {
}

shinyApp(ui = ui, server = server)

```
---

Now we will see **how to connect these widgets to reactive outputs so that objects update themselves whenever your user interacts with a widget**.

You can create a **reactive output** with a two step process:
First, add an R object to your user interface. 
Then, tell Shiny how to build the object in the server function. The object will be reactive (it will automatically change) if the code that builds it uses a widget's value. 
*Every time user changes the widget value, the reactive code  will run.*

Shiny provides a family of functions that turn R objects into outputs for your user interface. 
Each function creates a specific type of output.


| Output function  |   Creates |
|------------------|---------------|
| dataTableOutput  |   DataTable |
| htmlOutput       |   raw HTML |
| imageOutput      |   image |
| plotOutput       |   plot |
| tableOutput      |   table |
| textOutput       |   text |
| uiOutput         |   raw HTML |
| verbatimTextOutput |  text |



You can add output to the user interface by placing the output function inside sidebarPanel or mainPanel in the UI. 
For example, the UI object below uses **textOutput to add a reactive line of text to the main panel** of the Shiny App.

---
```R

ui <- fluidPage(
  titlePanel("censusVis"),
  
  sidebarLayout(
    sidebarPanel(
      helpText("Create demographic maps with 
               information from the 2010 US Census."),
      
      selectInput("var", 
                  label = "Choose a variable to display",
                  choices = c("Percent White", 
                              "Percent Black",
                              "Percent Hispanic", 
                              "Percent Asian"),
                  selected = "Percent White"), #default
      
      sliderInput("range", 
                  label = "Range of interest:",
                  min = 0, max = 100, value = c(0, 100))
    ),
    
    mainPanel(
      textOutput("selected_var") # the name of this output is "selected_var"
    )
  )
)

```
---

Notice that ```textOutput``` takes an argument, the character string "selected_var".
**Each of the output functions require a single argument:** a character string that Shiny will use as the name of your reactive element. 
Your users will not see this name, but you will use it later in the server code. 

Placing a function in UI tells Shiny where to **display** your object. Next, you need to tell Shiny how to **build** that object. 
We do this by providing the R code that builds the object in the server function.

The server function maintains a list-like object named ```output``` that contains all of the code needed to update the outputs in your app. 
Each output has its own entry in the list.

You can create an entry by defining a new element for output within the server function like below. 
The element name should **match** the name of the reactive element that you created in the ui.

**In the server function below, ```selected_var``` field of ```output``` matches ```textOutput("selected_var")``` in the UI.**

---
```
server <- function(input, output) {
  
  output$selected_var <- renderText({ 
    "You have selected this"              # send this text to the output in UI 
  })
  
}
```
---

Each entry to output should contain the output of one of Shiny’s render functions. 
These functions capture an R expression and do some light pre-processing on the expression. 
Use the render function that corrresponds to the type of reactive object you are making.


| render function  | creates   |
|------------------|-----------|
| renderDataTable  | DataTable |
| renderImage      | images (saved as a link to a source file) |
| renderPlot       | plots |
| renderPrint      | any printed output |
| renderTable      | data frame, matrix, other table like structures |
| renderText       | character strings |
| renderUI         | a Shiny tag object or HTML |


Each render function takes a single argument: an R expression surrounded by braces, {}. 
The expression can be one simple line of text, or it can involve many lines of code, as if it were a complicated function call.

If you run the app with the server function above, it will display “You have selected this” in the main panel. 
However, the text will not be reactive. It will not change even if you manipulate the widgets of your app.

**You can make the text reactive by asking Shiny to use a widget's value when it builds the text**. 
Let’s look at how to do this. 
Shiny will automatically make an object reactive if the object uses an input value. 
For example, the server function below creates a reactive line of text by calling the value of the select box widget to build the text.


---
```
server <- function(input, output) {
  
  output$selected_var <- renderText({ 
    paste("You have selected ", input$var)  # create a text message using the input from user. Now this is reactive. 
  })
  
}
```
---


Shiny tracks which outputs depend on which widgets. 
When a user changes a widget, 
Shiny will rebuild all of the outputs that depend on the widget, 
using the new value of the widget as it goes. 
As a result, the rebuilt objects will be completely up-to-date.

Let's see the complete code with some additional styling: 

In [ ]:
# Do not run this code cell, copy and paste it to app.R

library(shiny)

ui <- fluidPage(
  titlePanel("censusVis"),

  sidebarLayout(
    sidebarPanel(
      helpText("Create demographic maps with 
               information from the 2010 US Census."),

      selectInput("var", 
                  label = em("Choose a variable to display"),
                  choices = c("Percent White", 
                              "Percent Black",
                              "Percent Hispanic", 
                              "Percent Asian"),
                  selected = "Percent White"), #default

      sliderInput("range", 
                  label = em("Range of interest:"),
                  min = 0, max = 100, value = c(0, 100))
    ),

    mainPanel(
      h3("Output is displayed here:"),
      textOutput("selected_var"),
      p(),
      textOutput("selected_range")
        
    )
  )
)

server <- function(input, output) {

  output$selected_var <- renderText({ 
    paste("You have selected ", input$var)  # create a text message using the input from user. Now this is reactive. 
  })
  
  output$selected_range <- renderText({ 
    paste(" in this range: ", input$range[1], " - ", input$range[2]) 
  })
        
}


shinyApp(ui=ui, server=server)

** Copy and paste the above code into app.R under ```module2-widgets``` and deploy by running the following code here:**

In [ ]:

# RUN THIS CELL TO DEPLOY TO SHINY SERVER

dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module2-widgets" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

In a more realistic Shiny Apps, these values would be used to modify the visualization interactively. We will see an example of that in the practices.

